In [3]:
import torch
import torchvision
from torch import nn 
from torch.nn import functional as F
from d2l import torch as d2l

In [4]:
def cls_predictor(num_inputs, num_anchors, num_classes):
    return nn.Conv2d(num_inputs, num_anchors * (num_classes + 1), 
                                        kernel_size=3, padding=1)

In [5]:
def bbox_predictor(num_inputs, num_anchors):
    return nn.Conv2d(num_inputs, num_anchors * 4, kernel_size=3, padding=1)

In [6]:
def forward(x, block):
    return block(x)

Y1 = forward(torch.zeros((2, 8, 20, 20)), cls_predictor(8, 5, 10 ))
Y2 = forward(torch.zeros((2, 16, 10, 10)), cls_predictor(16, 3, 10))

Y1.shape, Y2.shape

(torch.Size([2, 55, 20, 20]), torch.Size([2, 33, 10, 10]))

In [7]:
def flatten_pred(pred):
    # 调整 数据顺序： 批次， h ， w， channels， 在平摊 后三个维度
    # 变成 [ size，h * w * channels ]
    return torch.flatten(pred.permute(0, 2, 3, 1), start_dim=1)

def concat_preds(preds):
        return torch.cat( [flatten_pred(p)  for p in preds], dim=1)

In [8]:
concat_preds([Y1, Y2]).shape

torch.Size([2, 25300])

In [9]:
def down_sample_blk(in_channels, out_channels):
    blk = []
    for _ in range(2):
        blk.append(nn.Conv2d(in_channels, out_channels, 
                             kernel_size=3,padding=1))
        blk.append(nn.BatchNorm2d(out_channels))
        blk.append(nn.ReLU())
        in_channels = out_channels
    blk.append(nn.MaxPool2d(2))
    return nn.Sequential(*blk)

In [10]:
forward(torch.zeros((2, 3, 20,20)), down_sample_blk(3, 10)).shape

torch.Size([2, 10, 10, 10])

In [11]:
# 基本网络
def base_net():
     blk = []
     num_filters = [3, 16, 32, 64]
     for  i in range(len(num_filters) - 1):
          blk.append( down_sample_blk(num_filters[i], num_filters[i+1]))
     return  nn.Sequential(*blk)


forward(torch.zeros((2, 3, 256, 256)), base_net()).shape

torch.Size([2, 64, 32, 32])

In [28]:
def get_block(i):
    if i==0:
        blk = base_net()
    elif i==1:
        blk = down_sample_blk(64, 128)
    elif i==4 :
        blk = nn.AdaptiveMaxPool2d((1, 1))
    else:
        blk = down_sample_blk(128 ,128)
    return blk

In [30]:
def blk_forward(X, blk, size, ratio, cls_predictor, bbox_predictor):
    Y = blk(X)
    anchors = d2l.multibox_prior(Y, size, ratio)
    cls_preds = cls_predictor(Y)
    bbox_preds = bbox_predictor(Y)

    return (Y, anchors, cls_preds, bbox_preds)

In [31]:
sizes = [ [ 0.2, 0.272], [0.37, 0.447], [0.54, 0.619], [0.71, 0.79],
          [0.88, 0.961]]
ratios = [[1, 2, 0.5]] * 5
num_anchors = len(sizes[0]) + len(ratios[0]) - 1

In [32]:
class TinySSD(nn.Module):
    def __init__(self, num_classes, **kwargs):
        super().__init__(**kwargs)
        self.num_classes =  num_classes
        id_to_in_channels = [64, 128, 128, 128, 128]
        for i in range(5):
            setattr(self, f'block_{i}',get_block(i))
            setattr(self, f'cls_{i}', cls_predictor(id_to_in_channels[i], 
                                                                        num_anchors, num_classes ))
            setattr(self, f'bbox_{i}', bbox_predictor(id_to_in_channels[i], num_anchors))

    def forward(self, x):
            anchors, cls_preds, bbox_preds = [None] * 5, [None] * 5, [None] * 5
            for i in range(5):
                 x , anchors[i], cls_preds[i], bbox_preds[i] = blk_forward(x, getattr(self, f'block_{i}'),
                                                                                                            sizes[i], ratios[i],getattr(self, f'cls_{i}'),
                                                                                                            getattr(self, f'bbox_{i}'))
            anchors = torch.cat(anchors, dim=1)
            cls_preds=concat_preds(cls_preds)
            cls_preds = cls_preds.reshape(cls_preds.shape[0], -1 , self.num_classes+1)
            bbox_preds = concat_preds(bbox_preds)
            return anchors, cls_preds, bbox_preds

In [33]:
net = TinySSD(num_classes = 1)

X =torch.zeros((32, 3, 256, 256))
anchors , cls_preds, bbox_preds = net(X)

anchors.shape, cls_preds.shape, bbox_preds.shape

(torch.Size([1, 5444, 4]), torch.Size([32, 5444, 2]), torch.Size([32, 21776]))

In [34]:
batch_size = 32
train_iter, _ = d2l.load_data_bananas(batch_size)


read 1000 training examples
read 100 validation examples


In [35]:
device , net = d2l.try_gpu(), TinySSD(num_classes=1)
trainer  = torch.optim.SGD(net.parameters(), lr=0.2, weight_decay=5e-4)

In [37]:
cls_loss = nn.CrossEntropyLoss(reduction='none')
bbox_loss = nn.L1Loss(reduction='none')

def clac_loss(cls_preds, cls_labels, bbox_preds, bbox_labels, bbox_masks):
    batch_size, num_classes = cls_preds.shape[0], cls_preds.shape[2]
    cls = cls_loss(cls_preds.reshape(-1 , num_classes), 
                    cls_labels.reshape(-1)).reshape(batch_size, -1).mean(dim=1)  
    bbox = bbox_loss(bbox_preds * bbox_masks, bbox_labels * bbox_masks).mean(dim=1)
    return cls + bbox

In [36]:
def cls_eval( cls_preds, cls_labels):
    # cls_preds shape  [batch_size, num_anchors, classes]
    return float(( cls_preds.argmax(dim= -1).type(cls_labels.dtype) == cls_labels).sum() )
def bbox_eval(bbox_preds, bbox_labels, bbox_masks):
    return float(( torch.abs((bbox_labels - bbox_preds) * bbox_masks)).sum() )

In [ ]:
num_epochs, timer = 20, d2l.Timer()

animator = d2l.Animator(xlabel='epoch', xlim=[1,num_epochs],
                                                        legend=[ 'class error', 'bbox mae'])

net = net.to(device)

for epoch in range(num_epochs):
    metric = d2l.Accumulator(4)
    net.train()
    for features, target in train_iter:
        timer.start()
        trainer.zero_grad()
        X, Y = features.to(device), target.to(device)

        anchors, cls_preds, bbox_preds = net(X)
        # 将真实标签分配给锚框
        bbox_labels , bbox_masks, cls_labels  =d2l.multibox_target(anchors, Y)
        l = clac_loss(cls_preds, cls_labels, bbox_preds, bbox_labels, bbox_masks)

        l.mean().backward()
        trainer.step()

        metric.add(cls_eval(cls_preds, cls_labels) , cls_labels.numel(), 
                                bbox_eval(bbox_preds, bbox_labels, bbox_masks),
                                bbox_labels.numel())
    cls_err, bbox_mae = 1 - metric[0] / metric[1], metric[2] / metric[3]
    animator.add(epoch+1, (cls_err,   bbox_mae))

print(f'class err   {cls_err:.2e}, bbox mae{bbox_mae :.2e}')
# mae  ： mean abs error , 平均绝对误差
print(f' {len(train_iter.dataset) / timer.stop() :.1f} examples /sec   on'
                f'{str(device)}')

![20250123114516](https://raw.githubusercontent.com/AustinSuun/image/main/img/20250123114516.png)

In [1]:
x = torchvision.io.read_image('../img/banana.jpg').unsqueeze(0).float()
img = x.squeeze(0).permute(1, 2, 0).long()

NameError: name 'torchvision' is not defined

In [ ]:
def predict(x):
    net.eval()
    anchors, cls_preds, bbox_preds = net(x.to(device))
    # cls_probs shape  [batch_size,  anchors_num, class_num]
    # 把一个锚框的 预测类 softmax
    cls_probs = F.softmax(cls_preds, dim=2).permute(0, 2, 1)
    output = d2l.multibox_detection(cls_probs, bbox_preds, anchors)
    idx = [i for i , row in enumerate(output[0]) if row[0]!= -1]
    return output[0 ,idx]

output = predict(x)

In [ ]:
def display(img, output, threshold):
    d2l.set_figsize((5, 5))
    fig =d2l.plt.imshow(img)
    for row in output:
        score = float(row[1])
        if score < threshold:
            continue
        h, w = img.shape[:2]
        bbox = [row[2:6] * torch.tensor((h, w, h, w), device=row.device)]
        d2l.show_bboxes(img.axes, bbox, '%.2f' % score, 'w')

display(img, output.cpu(), threshold=0.9)

![20250123213014](https://raw.githubusercontent.com/AustinSuun/image/main/img/20250123213014.png)